In [13]:
import pandas as pd

df = pd.read_json(path_or_buf='gg2013.json')
df.head(10)

data = df['text']

print(data[0])

JLo's dress! #eredcarpet #GoldenGlobes


In [14]:
import re

def cleanse(line):
    # replace everything to ' ' except whitespace, alphanumeric character, apostrophe, hashtag, @
    return re.sub(r'[^\w\s\'#@]', ' ', line)


def identify_entities(text, verbose=False):
    ents = list(nlp(text).ents)
    if verbose:
        tags = {}
        for ent in ents:    
            entity = ' '.join(t.orth_ for t in ent).strip()
            if entity not in tags:
                tags[entity]=[ent.label_]
        return tags
    else:
        return [' '.join(t.orth_ for t in ent).strip() for ent in ents]

In [15]:
import spacy
# python -m spacy download en
nlp = spacy.load('en')
doc = nlp(cleanse(data[0]))
print([(w.text, w.pos_) for w in doc])

[('JLo', 'PROPN'), ("'s", 'PART'), ('dress', 'NOUN'), (' ', 'SPACE'), ('#', 'SYM'), ('eredcarpet', 'NOUN'), ('#', 'SYM'), ('GoldenGlobes', 'PROPN')]


In [16]:
import spacy
# python -m spacy download en
nlp = spacy.load('en')
doc = nlp(cleanse(data[0]))
persons_rough = []
# names with frequency
persons_dic = {}

for d in data[:50]:
    doc = nlp(cleanse(d))
    # print(d)
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            name = ent.text
            if name in persons_dic:
                persons_dic[name] += 1
            else:
                persons_dic[name] = 1
            persons_rough.append(ent.text)
            print(ent.text)

print(persons_rough)
print(persons_dic)

Sofia Vergara's
Kerry Washington
Miu Miu
Anne Hathaway
Jennifer Lopez's
Al Roker  
Hugh Jackman
the Ben Affleck
Jennifer Lopez's
Jessica Alba
Kate Hudson's
Kristen Wiig
Hugh Jackman
Anne Hathaway
Al Roker
Leo DiCaprio
Kerry Washington
Ewan McGregor
Jennifer Lopez's
Lawd  
Helen Mirren
Miu Miu
@kerrywashington
Kerry Washington
Loved
Django Tour
Miu Miu
Natalie Morales
Awkward Savannah
Kerry Washington
Saw George Clooney         
Lo
Julianna Rancic
Ryan Seacrest
Lol  
Llega
Leonardo DiCaprio
Kerry Washington
J Lo
@whitbyfoodie
Hamm
Ben Affleck's
  Best
Kate
Rachel Weisz
The Deep Blue Sea
["Sofia Vergara's", 'Kerry Washington', 'Miu Miu', 'Anne Hathaway', "Jennifer Lopez's", 'Al Roker  ', 'Hugh Jackman', 'the Ben Affleck', "Jennifer Lopez's", 'Jessica Alba', "Kate Hudson's", 'Kristen Wiig', 'Hugh Jackman', 'Anne Hathaway', 'Al Roker', 'Leo DiCaprio', 'Kerry Washington', 'Ewan McGregor', "Jennifer Lopez's", 'Lawd  ', 'Helen Mirren', 'Miu Miu', '@kerrywashington', 'Kerry Washington', 'Loved

In [22]:
def find_nominees():
    pattern = re.compile("(win)|(won)|(lose)|(lost)|(nominat)", re.IGNORECASE)
    pattern1 = re.compile("(\saward\s)|(\sbest\s)", re.IGNORECASE)
    persons_dic = {}
    # awards_dic = {}
    for line in data[:5000]:
        line = cleanse(line)

        match = pattern.search(line)

        # add weight
        w = 1
        if match:
            w = 20
            # print(line)


        doc = nlp(line)
        # possible award name
        line_arr = line.lower().split(" ")
        award = ''
        if 'best' in line_arr:
            i = line_arr.index('best')
            award = line_arr[i:i+3]
            award = " ".join(award).rstrip()
            # print(award)

        if 'award' in line_arr:
            i = line_arr.index('award')
            award = line_arr[i-2:i+1]
            award = " ".join(award).rstrip()
            # print(award)


        for ent in doc.ents:
            if ent.label_ == 'PERSON':
                name = ent.text.strip()
                if name in persons_dic:
                    persons_dic[name] += w
                else:
                    persons_dic[name] = w
                # print(ent.text)

    # filter frequency <= 10
    nominees = {key: value for key, value in persons_dic.items() if value > 10}
    nominees = sorted(nominees.items(), key=lambda e: e[1], reverse=True)
    print(nominees)
    # print(persons_dic)
    print(len(nominees))


In [23]:
%%time
find_nominees()

[('Glenn Close', 200), ('Anne Hathaway', 186), ('Jennifer Lawrence', 182), ('Jessica Chastain', 150), ('Kerry Washington', 135), ('RT @PimpBillClinton', 120), ('Mani Cam', 100), ('Laura Linney', 100), ('Zac Posen', 84), ('Amy', 84), ('Amy Poehler', 82), ('Sofia Vergara', 80), ('Harry Winston', 80), ('Kate Hudson', 74), ('Oscar', 63), ('Ben Affleck', 61), ('RT @Zac_Posen', 60), ('Lucy Liu', 59), ('Miranda Kerr', 57), ('Halle Berry', 55), ('Helen Mirren', 54), ('RT @goldenglobes', 53), ('Al Roker', 52), ('@GoldenGlobes', 52), ('Jennifer Lopez', 52), ('Orlando Bloom', 52), ('Hugh Jackman', 51), ('Goldenglobes', 51), ('Denzel Washington', 50), ('Jessica Alba', 49), ('Leonardo DiCaprio', 49), ('Matt Lauer', 49), ('Leo', 47), ('Ewan McGregor', 46), ('Adele', 46), ('@goldenglobes', 46), ('Denzel', 44), ('Julianne Hough', 43), ('Nicole Richie', 42), ('Anjelica', 41), ('LO', 41), ('Barbara', 41), ('Taylor Swift', 40), ('Huston', 40), ('Emmy', 40), ('WIN', 40), ('OH BOO', 40), ('Suck', 40), ('So